In [1]:
import pandas as pd 
import dart_fss as dart
from constants import DART_API_KEY
dart.set_api_key(api_key=DART_API_KEY)

'b470732e660dd2fe2997205c7f59524d889713db'

In [3]:
def name2corp(corp_name):
	# get list of all corp
	# To be change DB
	corp_list = dart.get_corp_list()
	print("got corp list")

	# get corp class by corp name in corp_list
	corp_info = corp_list.find_by_corp_name(corp_name, exactly=True)[0]
	return corp_info

In [41]:
# multiple index columns to single index columns
def multiple2single(df, sheet_type):
	# multiple index to single index 
	df.columns = df.columns.to_flat_index()

	# set columns 
	new_cols = []
	for col in df.columns.values:
		if type(col[1]) == tuple:
			if sheet_type == 'bs':
				new_cols.append(col[0])
			else: #is or cf
				new_cols.append(col[0].split('-')[-1])
		else:
			new_cols.append(col[1]) 
	df.columns = new_cols

	return df

In [53]:
# select dataframe by column index value
def get_condition_row(df, val):
	class_list= [c for c in df.columns.values if 'class' in c]
	class_list.sort(reverse=True)

	for cl in class_list:
		df_sel = df.loc[df[cl] == val]
		if df_sel.empty:
			pass
		else:
			break
	
	return df_sel

# 1. Get financial sheet by DART

In [2]:
corp_list = dart.get_corp_list()

Output()

Output()

Output()

In [4]:
corp_name = "삼성전자"
corp_info = name2corp(corp_name)

got corp list


In [43]:
# fs = dart.fs.extract(corp_code=corp_code, bgn_de='20120101')
fs = corp_info.extract_fs(bgn_de='20210101')

Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

# 2. Split financial sheet to balance sheet, incom statement, cash flow

## 2-1 Balance sheet

In [44]:
df_bs = fs['bs']
df_bs = multiple2single(df_bs, 'bs')
df_bs.head()

,concept_id,label_ko,label_en,class0,class1,class2,class3,class4,20211231,20201231,20191231,20181231
0,ifrs-full_CurrentAssets,유동자산,Current assets,재무상태표 [abstract],자산 [abstract],유동자산,None,None,"218,163,185,000,000.0","198,215,579,000,000.0","181,385,260,000,000.0","174,697,424,000,000.0"
1,ifrs-full_CashAndCashEquivalents,현금및현금성자산,Cash and cash equivalents,재무상태표 [abstract],자산 [abstract],유동자산,현금및현금성자산,None,"39,031,415,000,000.0","29,382,578,000,000.0","26,885,999,000,000.0","30,340,505,000,000.0"
2,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,Short-term financial instruments,재무상태표 [abstract],자산 [abstract],유동자산,단기금융상품,None,"81,708,986,000,000.0","92,441,703,000,000.0","76,252,052,000,000.0","65,893,797,000,000.0"
3,entity00126380_udf_BS_201851017339116_CurrentA...,단기상각후원가금융자산,Short-term financial assets at amortized cost,재무상태표 [abstract],자산 [abstract],유동자산,단기상각후원가금융자산,None,"3,369,034,000,000.0","2,757,111,000,000.0","3,914,216,000,000.0","2,703,693,000,000.0"
4,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,단기당기손익-공정가치금융자산,Short-term financial assets at fair value thro...,재무상태표 [abstract],자산 [abstract],유동자산,단기당기손익-공정가치금융자산,None,"40,757,000,000.0","71,451,000,000.0","1,727,436,000,000.0","2,001,948,000,000.0"


In [55]:
df_sel = get_condition_row(df_bs, '현금및현금성자산')
df_sel.head()

,concept_id,label_ko,label_en,class0,class1,class2,class3,class4,20211231,20201231,20191231,20181231
1,ifrs-full_CashAndCashEquivalents,현금및현금성자산,Cash and cash equivalents,재무상태표 [abstract],자산 [abstract],유동자산,현금및현금성자산,None,"39,031,415,000,000.0","29,382,578,000,000.0","26,885,999,000,000.0","30,340,505,000,000.0"


## 2-2 Income statement

In [45]:
df_is = fs['is']
df_is = multiple2single(df_is, 'is')
df_is.head()

,concept_id,label_ko,label_en,class0,class1,class2,20211231,20201231,20191231,20181231
0,ifrs-full_Revenue,수익(매출액),Revenue(Sales),손익계산서 [abstract],수익(매출액),None,"279,604,799,000,000.0","236,806,988,000,000.0","230,400,881,000,000.0","243,771,415,000,000.0"
1,ifrs-full_CostOfSales,매출원가,Cost of sales,손익계산서 [abstract],매출원가,None,"166,411,342,000,000.0","144,488,296,000,000.0","147,239,549,000,000.0","132,394,411,000,000.0"
2,ifrs-full_GrossProfit,매출총이익,Gross profit,손익계산서 [abstract],매출총이익,None,"113,193,457,000,000.0","92,318,692,000,000.0","83,161,332,000,000.0","111,377,004,000,000.0"
3,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,Selling and administrative expenses,손익계산서 [abstract],판매비와관리비,None,"61,559,601,000,000.0","56,324,816,000,000.0","55,392,823,000,000.0","52,490,335,000,000.0"
4,dart_OperatingIncomeLoss,영업이익,Operating profit,손익계산서 [abstract],영업이익,None,"51,633,856,000,000.0","35,993,876,000,000.0","27,768,509,000,000.0","58,886,669,000,000.0"


## 2-3 Cash Flow

In [46]:
df_cf = fs['cf']
df_cf = multiple2single(df_cf, 'cf')
df_cf.head()

,concept_id,label_ko,label_en,class0,class1,class2,class3,20211231,20201231,20191231,20181231
0,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동 현금흐름,Cash flows from (used in) operating activities,현금흐름표 [abstract],영업활동 현금흐름,None,None,"65,105,448,000,000.0","65,287,009,000,000.0","45,382,915,000,000.0","67,031,863,000,000.0"
1,entity00126380_udf_CF_201710211099539_CashFlow...,영업에서 창출된 현금흐름,Cash generated from operations,현금흐름표 [abstract],영업활동 현금흐름,영업에서 창출된 현금흐름,None,"72,676,199,000,000.0","68,148,810,000,000.0","56,635,791,000,000.0","78,025,064,000,000.0"
2,entity00126380_udf_CF_201710211092510_udf_CF_2...,당기순이익,Profit (loss) for the year,현금흐름표 [abstract],영업활동 현금흐름,영업에서 창출된 현금흐름,당기순이익,"39,907,450,000,000.0","26,407,832,000,000.0","21,738,865,000,000.0","44,344,857,000,000.0"
3,entity00126380_udf_CF_2017102110929616_udf_CF_...,조정,Adjustments,현금흐름표 [abstract],영업활동 현금흐름,영업에서 창출된 현금흐름,조정,"49,055,633,000,000.0","41,618,554,000,000.0","37,442,682,000,000.0","43,604,573,000,000.0"
4,entity00126380_udf_CF_2017102110934928_udf_CF_...,영업활동으로 인한 자산부채의 변동,Changes in assets and liabilities arising from...,현금흐름표 [abstract],영업활동 현금흐름,영업에서 창출된 현금흐름,영업활동으로 인한 자산부채의 변동,"-16,286,884,000,000.0","122,424,000,000.0","-2,545,756,000,000.0","-9,924,366,000,000.0"
